# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [2]:
%help

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.38.1 



# Available Magic Commands

## Sessions Magic

----
    %help                             Return a list of descriptions and input types for all magic commands. 
    %profile            String        Specify a profile in your aws configuration to use as the credentials provider.
    %region             String        Specify the AWS region in which to initialize a session. 
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\ USERNAME \.aws\config" on Windows.
    %idle_timeout       Int           The number of minutes of inactivity after which a session will timeout. 
                                      Default: 2880 minutes (48 hours).
    %session_id_prefix  String        Define a String that will precede all session IDs in the format 
                                      [session_id_prefix]-[session_id]. If a session ID is not provided,
                                      a random UUID will be generated.
    %status                           Returns the status of the current Glue session including its duration, 
                                      configuration and executing user / role.
    %session_id                       Returns the session ID for the running session. 
    %list_sessions                    Lists all currently running sessions by ID.
    %stop_session                     Stops the current session.
    %glue_version       String        The version of Glue to be used by this session. 
                                      Currently, the only valid options are 2.0, 3.0 and 4.0. 
                                      Default: 2.0.
----

## Selecting Job Types

----
    %streaming          String        Sets the session type to Glue Streaming.
    %etl                String        Sets the session type to Glue ETL.
    %glue_ray           String        Sets the session type to Glue Ray.
----

## Glue Config Magic 
*(common across all job types)*

----

    %%configure         Dictionary    A json-formatted dictionary consisting of all configuration parameters for 
                                      a session. Each parameter can be specified here or through individual magics.
    %iam_role           String        Specify an IAM role ARN to execute your session with.
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\%USERNAME%\.aws\config` on Windows.
    %number_of_workers  int           The number of workers of a defined worker_type that are allocated 
                                      when a session runs.
                                      Default: 5.
    %additional_python_modules  List  Comma separated list of additional Python modules to include in your cluster 
                                      (can be from Pypi or S3).
    %%tags        Dictionary          Specify a json-formatted dictionary consisting of tags to use in the session.
----

                                      
## Magic for Spark Jobs (ETL & Streaming)

----
    %worker_type        String        Set the type of instances the session will use as workers. 
                                      ETL and Streaming support G.1X, G.2X, G.4X and G.8X. 
                                      Default: G.1X.
    %connections        List          Specify a comma separated list of connections to use in the session.
    %extra_py_files     List          Comma separated list of additional Python files From S3.
    %extra_jars         List          Comma separated list of additional Jars to include in the cluster.
    %spark_conf         String        Specify custom spark configurations for your session. 
                                      E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer
----
                                      
## Magic for Ray Job

----
    %min_workers        Int           The minimum number of workers that are allocated to a Ray job. 
                                      Default: 1.
    %object_memory_head Int           The percentage of free memory on the instance head node after a warm start. 
                                      Minimum: 0. Maximum: 100.
    %object_memory_worker Int         The percentage of free memory on the instance worker nodes after a warm start. 
                                      Minimum: 0. Maximum: 100.
----

## Action Magic

----

    %%sql               String        Run SQL code. All lines after the initial %%sql magic will be passed
                                      as part of the SQL code.  
----



####  Run this cell to set up and start your interactive session.


In [5]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

You are already connected to a glueetl session 4e3d3415-d618-4f4d-a5d1-2d8f9a384a17.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 4e3d3415-d618-4f4d-a5d1-2d8f9a384a17.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 3.0


You are already connected to a glueetl session 4e3d3415-d618-4f4d-a5d1-2d8f9a384a17.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 4e3d3415-d618-4f4d-a5d1-2d8f9a384a17.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



In [19]:
import pyspark
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Define the context

In [6]:
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

# Reading data from S3

In [42]:
dir_data_customers = r's3://app-planejamento-estrategico/data/tutorial_johnny/customers/customers.csv'

## Define the schema

In [43]:
custom_schema_customers = StructType([
    StructField("customerid", IntegerType(), True),
    StructField("firstname", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("fullname", StringType(), True)
])

## Reading the file

In [59]:
def describe_dataframe(data, n_rows=5):
    
    # VALIDATE IF DATA IS PYSPARK DATAFRAME
    if isinstance(data, pyspark.sql.DataFrame):
        
        print("PYSPARK DATAFRAME")
        print("-"*50)
        
        # SHOW DATA
        print("SHOWING DATA")
        data.show(n_rows)
        
        # COUNT ROWS
        print("COUNTING ROWS")
        print(data.count())
        
        # LIST COLUMNS
        print("LISTING COLUMNS")
        print(data.columns)
        
        # DESCRIBING DATA
        print("DESCRIBING DATA")
        data.describe().show()
        
    elif isinstance(data, pd.DataFrame):
        
        print("PANDAS DATAFRAME")
        print("-"*50)
        
        # SHOW DATA
        print("SHOWING DATA")
        print(data.head(n_rows))
        
        # COUNT ROWS
        print("COUNTING ROWS")
        print(len(data))
        
        # LIST COLUMNS
        print("LISTING COLUMNS")
        print(data.columns)
        
        # DESCRIBING DATA
        print("DESCRIBING DATA")
        print(data.describe())

## SPARK

In [45]:
df_customers = spark.read.csv(dir_data_customers, 
                              header=True, 
                              schema=custom_schema_customers)

In [57]:
describe_dataframe(data=df_customers)

PYSPARK DATAFRAME
--------------------------------------------------
SHOWING DATA
+----------+---------+-----------+----------------+
|customerid|firstname|   lastname|        fullname|
+----------+---------+-----------+----------------+
|       295|      Kim|Abercrombie| Kim Abercrombie|
|       297| Humberto|    Acevedo|Humberto Acevedo|
|       291|  Gustavo|     Achong|  Gustavo Achong|
|       299|    Pilar|   Ackerman|  Pilar Ackerman|
|       305|    Carla|      Adams|     Carla Adams|
+----------+---------+-----------+----------------+
only showing top 5 rows

COUNTING ROWS
634
LISTING COLUMNS
['customerid', 'firstname', 'lastname', 'fullname']
DESCRIBING DATA
+-------+------------------+---------+-----------+------------+
|summary|        customerid|firstname|   lastname|    fullname|
+-------+------------------+---------+-----------+------------+
|  count|               634|      634|        634|         634|
|   mean|1039.7917981072555|     null|       null|        null|
| s

## PANDAS

In [63]:
column_names_customers = [field.name for field in custom_schema_customers.fields]

In [64]:
df_pandas_customers = pd.read_csv(dir_data_customers, 
                                  header='infer', 
                                  names=column_names_customers)

In [65]:
describe_dataframe(data=df_pandas_customers)

PANDAS DATAFRAME
--------------------------------------------------
SHOWING DATA
   customerid  firstname     lastname          fullname
0         293  Catherine         Abel    Catherine Abel
1         295        Kim  Abercrombie   Kim Abercrombie
2         297   Humberto      Acevedo  Humberto Acevedo
3         291    Gustavo       Achong    Gustavo Achong
4         299      Pilar     Ackerman    Pilar Ackerman
COUNTING ROWS
635
LISTING COLUMNS
Index(['customerid', 'firstname', 'lastname', 'fullname'], dtype='object')
DESCRIBING DATA
        customerid
count   635.000000
mean   1038.615748
std     474.257783
min     291.000000
25%     654.000000
50%     993.000000
75%    1340.000000
max    1993.000000
